In [705]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime
from datetime import timedelta

# Imports and feature selection

In [706]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds_full = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')

In [707]:
# drop non-essential columns
docs.drop(['Middle Name','Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [708]:
# grab essential columns for building classifier
indem = claims[['Indemnity_Paid','ProviderId']]

In [709]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [710]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [711]:
# def build_classifier_label(item):
#     if item == 0:
#         return 'suit, no indem'
#     if item != 0:
#         return 'suit, indem'

def build_classifier_label(item):
    if item == 0:
        return 1
    if item != 0:
        return 2

In [712]:
indem['indem_label'] = indem['Indemnity_Paid'].apply(lambda item: build_classifier_label(item))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Join docs and indemnity tables, fill NaNs with 'no suit'

In [713]:
joined = pd.merge(docs, indem[['ProviderId', 'indem_label']], how='left', on='ProviderId')
joined.drop_duplicates(inplace=True)

In [714]:
# finish indemnity classifier
# find NaNs in indem_label column, replace with 'no suit'
def indem_label_no_suit(x):
    if type(x) == np.float_: 
        if np.isnan(x): 
#             return 'no suit'
            return 0
        else:
            return x

In [715]:
joined.indem_label = joined.indem_label.apply(lambda item: indem_label_no_suit(item))

# Claims data
#### building classifiers for BoardCredit and ClaimsBand, then merging with joined

In [717]:
def build_classifier(item):
    if item == 0:
        return 0
    if item != 0:
        return 1

In [718]:
# resave debs_and_creds with just the desired columns
# NOTE: typo 'RecordKepping'
debs_and_creds = debs_and_creds_full[['ProviderId','BoardCredit','ClaimsBand','Procedures','JCAHO','RecordKepping']]

# lower Provider Id
# reset index required
debs_and_creds['ProviderId'] = debs_and_creds['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())
debs_and_creds.tail()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ProviderId,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping
8458,19d9c4e6-e820-41c4-b398-134e6c0ba410,-55,-220,0,0,0
8459,6398eeea-10af-448f-bd10-364be1d0c90c,0,-8951,0,-2984,0
8460,d54c0b59-08e5-4ed7-ba5a-413b2016b269,-1366,0,0,-1366,-2733
8461,3eacedf6-9173-4a64-89c7-ba8b6e150850,-88,0,0,0,0
8462,ea83f592-67dd-46d5-b15c-9d0738f81c93,0,-258,0,0,0


In [719]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()
debs_and_creds.reset_index(inplace=True)

In [720]:
# use build_classifier to create binary column from BoardCredit & ClaimsBand
debs_and_creds['BoardCredit'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['ClaimsBand'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))
debs_and_creds['Procedures'] = debs_and_creds['Procedures'].apply(lambda item: build_classifier(item))
debs_and_creds['JCAHO'] = debs_and_creds['JCAHO'].apply(lambda item: build_classifier(item))
debs_and_creds['RecordKepping'] = debs_and_creds['RecordKepping'].apply(lambda item: build_classifier(item))

In [721]:
joined = pd.merge(joined, debs_and_creds, how='left', on='ProviderId')

# EDA

In [722]:
# print num of doctors in each bucket, using the 
print 'Doctors wih no suits: ', joined[joined['indem_label'] == 0].sort(['Last Name']).shape[0]
print 'Doctors with suits, no indemnity paid: ', joined[joined['indem_label'] == 1].sort(['Last Name']).shape[0]
print 'Doctors with suits, indemnity paid: ', joined[joined['indem_label'] == 2].sort(['Last Name']).shape[0]

Doctors wih no suits:  6147
Doctors with suits, no indemnity paid:  281
Doctors with suits, indemnity paid:  41


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# Feature engineering

### ~ build age column ~

In [723]:
# convert DoB to datetime object
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [724]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [725]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [726]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [727]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [728]:
# function to convert years into days
def to_years(dt):
    if dt:
        dt = (dt.days) / 365.25
    return dt

In [729]:
# apply to_years function to age column
joined['age'] = (joined['age'].apply(lambda dt: to_years(dt))).round()

In [730]:
# refill Date of Birth with NaNs
joined['Date of Birth'].replace(datetime(2060, 1, 1, 0, 0), np.nan, inplace=True)

In [731]:
def refill_age(x):
    if x < 0:
        x = np.nan
    return x

In [732]:
# refill age with nan's for docs with missing birthdate
joined['age'] = (joined['age'].apply(lambda dt: refill_age(dt))).round()

In [733]:
joined.corr()

,indem_label,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping,age
indem_label,1.000000,0.131730,0.114054,0.157412,0.191448,0.048830,0.113900
BoardCredit,0.131730,1.000000,0.525431,0.034711,0.434245,0.171050,0.109534
ClaimsBand,0.114054,0.525431,1.000000,0.055314,0.445487,0.224323,0.350789
Procedures,0.157412,0.034711,0.055314,1.000000,-0.001525,-0.032338,0.048823
JCAHO,0.191448,0.434245,0.445487,-0.001525,1.000000,0.269885,0.062431
RecordKepping,0.048830,0.171050,0.224323,-0.032338,0.269885,1.000000,0.003570
age,0.113900,0.109534,0.350789,0.048823,0.062431,0.003570,1.000000
